In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import math
import tqdm
from PyPDF2 import PdfMerger
import os
from os import listdir
from os.path import isfile, join

2023-05-02 12:18:09.592715: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 12:18:10.231886: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
wireplane = 'Z'

In [3]:
x_test_old = np.load('./Neutrino-Trained/processed_data/x_test_' + wireplane + '.npy')
y_test_old = np.load('./Neutrino-Trained/processed_data/y_test_AE_' + wireplane + '.npy')


mean_old = np.load('./Neutrino-Trained/AutoEncoder/results/'+wireplane+'/models/mean_AE_'+wireplane+'.npy')
std_old = np.load('./Neutrino-Trained/AutoEncoder/results/'+wireplane+'/models/std_AE_'+wireplane+'.npy')

x_test_scaled_old = (x_test_old-mean_old)/std_old
y_test_scaled_old = (y_test_old-mean_old)/std_old

In [4]:
x_test_new = np.load('./Neutrino-Trained-New-Dataset/processed_data/x_test_' + wireplane + '.npy')
y_test_new = np.load('./Neutrino-Trained-New-Dataset/processed_data/y_test_AE_' + wireplane + '.npy')


mean_new = np.load('./Neutrino-Trained-New-Dataset/AutoEncoder-Custom-Loss/results/'+wireplane+'/models/mean_AE_'+wireplane+'.npy')
std_new = np.load('./Neutrino-Trained-New-Dataset/AutoEncoder-Custom-Loss/results/'+wireplane+'/models/std_AE_'+wireplane+'.npy')

x_test_scaled_new = (x_test_new-mean_new)/std_new
y_test_scaled_new = (y_test_new-mean_new)/std_new

In [5]:
def custom_mse2(y_true, y_pred):

    return -1

In [6]:
ae128_old = load_model('./Neutrino-Trained/AutoEncoder/results/' + wireplane + '/models/batch_size128_CHECK_' + wireplane + 'plane_nu.h5', custom_objects={'custom_mse2':custom_mse2})
ae128_new = load_model('./Neutrino-Trained-New-Dataset/AutoEncoder-Custom-Loss/results/' + wireplane + '/models/batch_size128_' + wireplane + 'plane_nu.h5', custom_objects={'custom_mse2':custom_mse2})
ae_128_new_mse = load_model('./Neutrino-Trained-New-Dataset/AE_models/model_AE_128_mse' + wireplane + 'plane_nu.h5')

2023-05-02 12:18:11.976521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 12:18:11.991271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 12:18:11.991338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 12:18:11.993453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 12:18:11.993505: I tensorflow/compile

In [7]:
pred_old_old = ae128_old.predict(x_test_scaled_old, batch_size=4096)
pred_old_old = pred_old_old.reshape(pred_old_old.shape[0], pred_old_old.shape[1])


pred_new_new = ae128_new.predict(x_test_scaled_new, batch_size=4096)
pred_new_new = pred_new_new.reshape(pred_new_new.shape[0], pred_new_new.shape[1])

pred_mse = ae_128_new_mse.predict(x_test_scaled_new, batch_size=4096)
pred_mse = pred_mse.reshape(pred_mse.shape[0], pred_mse.shape[1])



2023-05-02 12:18:14.491536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-02 12:18:15.161190: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


25/25 [==============================] - 0s 3ms/step


In [8]:
from scipy import stats
x_testold_ = std_old*x_test_scaled_old+mean_old
y_testold_ = y_test_scaled_old*std_old + mean_old

x_testnew_ = std_new*x_test_scaled_new+mean_new
y_testnew_ = y_test_scaled_new*std_new + mean_new


pred_oo = pred_old_old*std_old + mean_old

pred_nn = pred_new_new*std_new + mean_new
pred_mse_ = pred_mse*std_new + mean_new


#pred1024 = predictions1024*std + mean

In [9]:

for idx in range(3000):
    if sum(abs(y_test_old[idx])) != 0 and sum(abs(y_test_new[idx])) != 0 and max(abs(y_test_old[idx])) < 20 and max(abs(y_test_new[idx])) < 20:
        fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 6))
        ax1.plot(y_testold_[idx], color = 'blue', label='truth')
        ax1.plot(pred_oo[idx], color = 'r', alpha=0.7, label='pred custom mse')
        ax1.set_title('trained on old:test on old')
        ax1.legend()

        ax2.plot(y_testnew_[idx], color = 'blue')
        ax2.plot(pred_nn[idx], color = 'r', alpha=0.7)
        ax2.plot(pred_mse_[idx], color = 'g', alpha=0.5, label='keras mse')
        ax2.set_title('trained on new:test on new')
        ax2.legend()

        for ax in fig.get_axes():
            ax.label_outer()
        plt.savefig('tmp/test' + str(idx) +  '.pdf',
            dpi=300,
            bbox_inches='tight', pad_inches=0.75)
        plt.close()

In [10]:
merger = PdfMerger()
path = 'tmp/'
pdf_files = [path+f for f in listdir(path) if (isfile(join(path, f)))]
print(pdf_files)

['tmp/test2632.pdf', 'tmp/test183.pdf', 'tmp/test745.pdf', 'tmp/test730.pdf', 'tmp/test1939.pdf', 'tmp/test1125.pdf', 'tmp/test748.pdf', 'tmp/test2280.pdf', 'tmp/test1784.pdf', 'tmp/test677.pdf', 'tmp/test1677.pdf', 'tmp/test1274.pdf', 'tmp/test1973.pdf', 'tmp/test300.pdf', 'tmp/test588.pdf', 'tmp/test1049.pdf', 'tmp/test746.pdf', 'tmp/test1818.pdf', 'tmp/test1545.pdf', 'tmp/test2499.pdf', 'tmp/test452.pdf', 'tmp/test263.pdf', 'tmp/test348.pdf', 'tmp/test2171.pdf', 'tmp/test1284.pdf', 'tmp/test1727.pdf', 'tmp/test1141.pdf', 'tmp/test1138.pdf', 'tmp/test2330.pdf', 'tmp/test1704.pdf', 'tmp/test1199.pdf', 'tmp/test1223.pdf', 'tmp/test389.pdf', 'tmp/test1469.pdf', 'tmp/test1423.pdf', 'tmp/test2618.pdf', 'tmp/test651.pdf', 'tmp/test2649.pdf', 'tmp/test1098.pdf', 'tmp/test1788.pdf', 'tmp/test2600.pdf', 'tmp/test2364.pdf', 'tmp/test1780.pdf', 'tmp/test632.pdf', 'tmp/test1880.pdf', 'tmp/test1133.pdf', 'tmp/test2598.pdf', 'tmp/test2292.pdf', 'tmp/test2864.pdf', 'tmp/test405.pdf', 'tmp/test457.p

In [11]:
for pdf_file in pdf_files:
    #Append PDF files
    merger.append(pdf_file)
#merger.write('pdfs/plts_tmp/plts_' + wireplane + '_cnn_'+str(int(min_cnn*100)) + '-' + str(int(max_cnn*100)) + '_' + str(num_pages) +  'pages.pdf')
merger.write('signals_lt_20_adc'+wireplane+'.pdf')
merger.close()

for file in pdf_files:
    os.remove(file)